In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import json
import os

def process_json_to_df(json_file):
    """Convert JSON weather data to pandas DataFrame"""
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    rows = []
    for day in data['data']['weather']:
        date = day['date']
        for hour in day['hourly']:
            row = {
                'date': date,
                'time': hour['time'],
                'tempC': float(hour['tempC']),
                'windspeedKmph': float(hour['windspeedKmph']),
                'humidity': float(hour['humidity']),
                'cloudcover': float(hour['cloudcover']),
                'precipMM': float(hour['precipMM']),
                'pressure': float(hour['pressure']),
                'visibility': float(hour['visibility']),
                'FeelsLikeC': float(hour['FeelsLikeC']),
                'uvIndex': float(hour['uvIndex'])
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

def prepare_features(df):
    """Prepare features for model training"""
    # Convert date and time
    df['date'] = pd.to_datetime(df['date'])
    df['hour'] = df['time'].astype(int) // 100
    
    # Extract time features
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    
    # Add daylight feature (simplified)
    df['is_daylight'] = ((df['hour'] >= 6) & (df['hour'] <= 18)).astype(int)
    df['daylight_hours'] = 12  # Simplified for Jakarta
    
    # Define features
    features = [
        'month', 'day', 'day_of_week', 'hour',
        'tempC', 'windspeedKmph', 'humidity',
        'cloudcover', 'precipMM', 'pressure',
        'visibility', 'FeelsLikeC', 'is_daylight',
        'daylight_hours'
    ]
    
    return df, features

def train_model(json_file):
    """Train model and save artifacts using TensorFlow"""
    # Create model directory if it doesn't exist
    if not os.path.exists('model'):
        os.makedirs('model')
    
    # Process data
    df = process_json_to_df(json_file)
    df, features = prepare_features(df)
    
    # Prepare features and target
    X = df[features]
    y = df['uvIndex']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build TensorFlow model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Single output for UV Index prediction
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    # Train the model
    model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)
    
    # Save the model and scaler
    model.save('model/uv_model_tf.h5')
    joblib.dump(scaler, 'model/scaler.pkl')
    
    print("TensorFlow model trained and saved successfully!")

if __name__ == "__main__":
    train_model('2022-2021-hourly-jakarta.json')
